In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from matplotlib.figure import Figure
import ipywidgets as widgets

%matplotlib inline

In [ ]:
''' Handling which data file to use for analysis,
    Selecting if the left or right tile was tested
'''

# TODO - file chooser
lpd_file_name = '/data/lpd/matt/' + 'lpdData-04764.hdf5_000001.h5'
lpd_file = h5py.File(lpd_file_name, 'r')
lpd_data = lpd_file['data'][()]

# Setup variables
num_images = len(lpd_data)
# Top left corner of a right hand tile
tile_start = [96, 0]

# TODO - choose whether run is for left or right tile, MINI 1 connector will be used 99% of the time, though maybe give
#        an option to select a different connector if need be? So default on 1 with options for the others
tile_choice = widgets.RadioButtons(
    options = ['Left Tile', 'Right Tile'],
    disabled = False
)
tile_choice_label = widgets.Label(
    value = 'Tile Orientation:',
    layout = widgets.Layout(width = '10%'),
)

# Event handling for radio buttons
def on_change(change):
    # Avoid getting property lock etc.
    if change['type'] == 'change' and change['name'] == 'value':
        tile_choice.value = change['new']   

tile_choice.observe(on_change)

# Mini connector slider
mini_connector_selector = widgets.IntSlider(
    # TODO - Change default value to 1 - currently at 4 based on test data
    value = 4,
    min = 1,
    max = 8,
    step = 1,
    orientation = 'horizontal'
)
mini_connector_label = widgets.Label(
    value = 'Mini Connector:',
    layout = widgets.Layout(width = '10%')
)

# TODO - Implement analyse_button
analyse_button = widgets.Button(
    description='Analyse Data',
    layout = widgets.Layout(width='20%')
)

def on_button_clicked(b):
    pass

analyse_button.on_click(on_button_clicked)

# Display widgets
display(widgets.HBox([tile_choice_label, tile_choice, mini_connector_label, mini_connector_selector]))
display(analyse_button)

In [ ]:
''' Setting up arrays for tiles
'''

def tile_array_setup():
    ''' Used to set variables defining where to get the data from lpd_data
    ''' 

    # Left or right tile
    if tile_choice.value == "Left Tile":
        tile_start[1] = 128
    else:
        tile_start[1] = 0
        
    # Tiles have a 32 pixel height
    tile_start[0] = 32 * (mini_connector_selector.value - 1)

In [ ]:
''' First entire image from data file with output plot
'''

single_image = lpd_data[:1,:,:]
single_image = np.reshape(single_image, (256, 256))

display_plot(single_image)

In [ ]:
''' First image of only the tile inspected with output plot
'''

tile_array_setup()

first_tile = lpd_data[:1,tile_start[0]:tile_start[0] + 32, tile_start[1]:tile_start[1] + 128]
first_tile = np.reshape(first_tile, (32, 128))

# Display plot
display_plot(first_tile)

In [ ]:
''' Creating a mean image based on all the images in the file, but just the size of the tile being tested
'''

tile_array_setup()

def display_mean_tile():
    mean_tile = get_mean_tile()
    display_plot(mean_tile)
    
def get_mean_tile():
    tile_data = total_tile()
    mean_tile = np.mean(tile_data, axis=0)
    return mean_tile

def total_tile():
    total_tile_data = lpd_data[:lpd_data.shape[0], tile_start[0]:tile_start[0] + 32, tile_start[1]:tile_start[1] + 128]
    return total_tile_data

display_mean_tile()

In [ ]:
''' Displays plots of numpy arrays, function can be used for whole images and tiles
'''

def display_plot(data):  
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    plt.axis('off')
    image = ax.imshow(data, cmap='ocean')

    # Create and add colorbar
    c_ticks = [0, 511, 1023, 1535, 2047, 2559, 3071, 3583, 4095]
    cbar = fig.add_axes([1.0, 0.30, 0.05, 0.4])
    colorbar = plt.colorbar(image, cax=cbar)
    colorbar.set_ticks(ticks=c_ticks)
    
    rows = data.shape[0]
    cols = data.shape[1]
    # Add vertical and horizontal lines to differentiate between chips and tiles
    for i in range(16, cols, 16):
        ax.vlines(i-0.5, 0, rows-1, color='k', linestyles='solid')
        # Add vertical lines to differentiate between tiles
        ax.vlines(128-0.5, 0, rows-1, color='k', linestyle='solid')

    for i in range(32, rows, 32):
        ax.hlines(i-0.5, 0, rows-1, color='k', linestyles='solid')

    plt.show()

In [ ]:
''' Standard deviation image for the entire image
'''
lpd_data_std_dev = np.std(lpd_data, axis=0)
display_plot(lpd_data_std_dev)

In [ ]:
''' Standard deviation data for a tile
'''

def get_stdev_tile():
    tile_data = total_tile()
    tile_std_dev = np.std(tile_data, axis=0)
    #np.clip(tile_std_dev, 0, 30, out=tile_std_dev)
    return tile_std_dev

tile_std_dev = get_stdev_tile()
display_plot(tile_std_dev)

In [ ]:
''' Histogram using average tile data
'''

mean_tile = get_mean_tile()

fig, ax = plt.subplots()
ax.hist(mean_tile.flatten(), bins=250)
    
plt.show()

In [ ]:
''' Histogram using standard deviation data of a tile
'''

stdev_tile = get_stdev_tile()

fig, ax = plt.subplots()
ax.hist(stdev_tile.flatten(), bins=250)
    
plt.show()

In [ ]:
''' Test for a bad chip within the tile
'''

# Bad chip: Average value of a chip - different to the others or different to a preset value

# Bad pixels: Average value of a pixel (per tile or per chip?) - is the value relative to others or a preset value

# Bad columns: Need to define pass/failure here

# Noise performance: Better to be done using a standard dev? Need to ask questions about this

mean_tile = get_mean_tile()
display_plot(mean_tile)
'''
print(np.amin(mean_tile))
q25, q75 = np.percentile(mean_tile, [25, 75])
iqr = q75 - q25
print("LQ: {} UQ: {}".format(q25, q75))
for i in range(0, len(mean_tile)):
    for j in range(0, len(mean_tile[i])):
        if mean_tile[i][j] > q75 + (3.0 * iqr):
            print(mean_tile[i][j] ,"U")
        if mean_tile[i][j] < q25 - (3.0 * iqr):
            print(mean_tile[i][j], "L")
'''

pixel_threshold = [3000, 3800]
bad_pixel_count = 0

for i in range(0, len(mean_tile)):
    for j in range(0, len(mean_tile[i])):
        if mean_tile[i][j] < pixel_threshold[0]:
            #print(mean_tile[i][j], "L")
            bad_pixel_count += 1
        elif mean_tile[i][j] > pixel_threshold[1]:
            #print(mean_tile[i][j], "U")
            bad_pixel_count += 1

print("Number of bad pixels: {}".format(bad_pixel_count))
print("Percentage of bad pixels: {}".format(bad_pixel_count / (32 * 128) * 100))

In [ ]:
import test_file

test_file.testing()
